In [1]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from IPython.display import clear_output,display

In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

import time

In [3]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '928902@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:01:07 GMT, Sun, 08 Jun 2025 15:01:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '134', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"928902@example.com","name":"test","image_url":null,"max_concurrent_bots":1,"id":6,"created_at":"2025-06-08T15:01:08.572383"}...


In [4]:

user_id = new_user['id']
user_id
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)


DEBUG: Making POST request to http://localhost:18056/admin/users/6/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:01:08 GMT, Sun, 08 Jun 2025 15:01:08 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '113', 'content-type': 'application/json'}
DEBUG: Response content: {"user_id":6,"id":6,"token":"aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw","created_at":"2025-06-08T15:01:09.448930"}...


In [21]:

meeting_url="https://meet.google.com/xdy-txjo-fyh"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="Vexa",
  #  language='es',
    task = 'transcribe'
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'xdy-txjo-fyh', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:05:07 GMT, Sun, 08 Jun 2025 15:05:06 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '401', 'content-type': 'application/json'}
DEBUG: Response content: {"id":6,"user_id":6,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"active","bot_container_id":"238281dc7fcfb4cfa13ae00daba80335684fca1b2fddd2c00f447c83f63ff718","start_time":"2025-06-08T15:05:08.057704","end_time":null,"data":{},"created_at":"2025-06-08T15:05:07.715109","updated_at":"2025-06-08T15:05:07.731955"}...


In [22]:
user_id

6

In [23]:
while True:
    transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
    df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
    clear_output()
    display(df.sort_values('absolute_start_time').tail(10))
    time.sleep(1)

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,3.04,"3,4,5,6,7,8,9,10,11,12",en,None,Dmitriy Grankin,2025-06-08T15:05:25.687109+00:00,2025-06-08T15:05:28.727109+00:00


KeyboardInterrupt: 

In [24]:
client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:05:34 GMT, Sun, 08 Jun 2025 15:05:34 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...


{'message': 'Stop request accepted and is being processed.'}

In [25]:
user_id

6

In [26]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:05:36 GMT, Sun, 08 Jun 2025 15:05:36 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '412', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"238281dc7fcfb4cfa13ae00daba80335684fca1b2fddd2c00f447c83f63ff718","container_name":"vexa-bot-6-e49f0c70","platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","status":"Up 28 seconds","created_at":"2025-06-08T15:05:07+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"6"},"meeting_id_from_name":"6"}]}...


In [49]:
client.update_bot_config(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    language='en',
    task = 'transcribe'
)


DEBUG: Making PUT request to http://localhost:18056/bots/google_meet/icj-aejh-tac/config
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'VLZbS0Vh3nNBNk8woojDWBG1r56cwP5PEIdPM74s'}
DEBUG: Params: None
DEBUG: JSON data: {'language': 'en', 'task': 'transcribe'}
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 17:48:37 GMT, Wed, 04 Jun 2025 17:48:38 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Reconfiguration request accepted and sent to the bot."}...


{'message': 'Reconfiguration request accepted and sent to the bot.'}

In [11]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JNPTl5QE0E23vHBLUH6cV1T9MkxBf2RwIauNSrMU'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 10:15:54 GMT, Sun, 08 Jun 2025 10:15:54 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '19', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[]}...


In [30]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:05:54 GMT, Sun, 08 Jun 2025 15:05:54 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '968', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":6,"user_id":6,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"completed","bot_container_id":"238281dc7fcfb4cfa13ae00daba80335684fca1b2fddd2c00f447c83f63ff718","start_time":"2025-06-08T15:05:08.057704","end_time":"2025-06-08T15:05:36.148599","data":{"name":"Test Meeting","languages":["en"],"participants":["Dmitriy Grankin"]},"created_at":"2025-06-08T15:05:07.715109","updated_at":"2025-06-08T1...


In [31]:
meetings

[{'id': 6,
  'user_id': 6,
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
  'status': 'completed',
  'bot_container_id': '238281dc7fcfb4cfa13ae00daba80335684fca1b2fddd2c00f447c83f63ff718',
  'start_time': '2025-06-08T15:05:08.057704',
  'end_time': '2025-06-08T15:05:36.148599',
  'data': {'name': 'Test Meeting',
   'languages': ['en'],
   'participants': ['Dmitriy Grankin']},
  'created_at': '2025-06-08T15:05:07.715109',
  'updated_at': '2025-06-08T15:05:51.617287'},
 {'id': 5,
  'user_id': 6,
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
  'status': 'completed',
  'bot_container_id': '7edd0e3d1b1aa9f8f668a0c84447889c36a8c4cb974ca15b80654d00c9a083bf',
  'start_time': '2025-06-08T15:01:10.681410',
  'end_time': '2025-06-08T15:02:05.246022',
  'data': {'name': 'Test Meeting'},
  'created_at': '2025-06-08T15:01

In [33]:
client.update_meeting_data(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    name="Test Meeting",
)


DEBUG: Making PATCH request to http://localhost:18056/meetings/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw'}
DEBUG: Params: None
DEBUG: JSON data: {'data': {'name': 'Test Meeting'}}
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:06:06 GMT, Sun, 08 Jun 2025 15:06:06 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '449', 'content-type': 'application/json'}
DEBUG: Response content: {"id":5,"user_id":6,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"completed","bot_container_id":"7edd0e3d1b1aa9f8f668a0c84447889c36a8c4cb974ca15b80654d00c9a083bf","start_time":"2025-06-08T15:01:10.681410","end_time":"2025-06-08T15:02:05.246022","data":{"name":"Test Meeting"},"created_at":"2025-06-08T15:01:10.280791","updated_at":"2025-06-08T15:06:07.017651"}...


{'id': 5,
 'user_id': 6,
 'platform': 'google_meet',
 'native_meeting_id': 'xdy-txjo-fyh',
 'constructed_meeting_url': 'https://meet.google.com/xdy-txjo-fyh',
 'status': 'completed',
 'bot_container_id': '7edd0e3d1b1aa9f8f668a0c84447889c36a8c4cb974ca15b80654d00c9a083bf',
 'start_time': '2025-06-08T15:01:10.681410',
 'end_time': '2025-06-08T15:02:05.246022',
 'data': {'name': 'Test Meeting'},
 'created_at': '2025-06-08T15:01:10.280791',
 'updated_at': '2025-06-08T15:06:07.017651'}

In [32]:
client.delete_meeting(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
)


DEBUG: Making DELETE request to http://localhost:18056/meetings/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'aQGEy4cX1XN9l60QOEsWKcWWKNNc0prjYUu7pkhw'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sun, 08 Jun 2025 15:06:02 GMT, Sun, 08 Jun 2025 15:06:02 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '88', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Meeting google_meet/xdy-txjo-fyh and all its transcripts have been deleted"}...


{'message': 'Meeting google_meet/xdy-txjo-fyh and all its transcripts have been deleted'}